In [1]:
# ===========================
# BLOCK 1: IMPORTS & CONFIG
# ===========================
import os
import cv2
import mediapipe as mp
from PIL import Image, ImageOps
import numpy as np

# ---- Path config ----
BASE_DIR = os.getcwd()
RAW_DIR = os.path.join(BASE_DIR, "./data/Train")
PROCESSED_DIR = os.path.join(BASE_DIR, "./data/Train_Cropped")

# ---- Preprocessing config ----
TARGET_SIZE = (384, 384)  # final face size
MARGIN_RATIO = 0.15  # extra padding around detected face
MAX_DIM = 1600  # max side length when downscaling
MIN_DIM = 256  # min side length when upscaling
USE_CENTER_FALLBACK = True  # use center crop if detection fails
USE_FACE_ALIGNMENT = True  # enable face alignment

# ---- MediaPipe Face Detection & Face Mesh ----
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

face_detection = mp_face_detection.FaceDetection(
    model_selection=1,  # 0: short range, 1: full range
    min_detection_confidence=0.5  # confidence threshold
)

face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)


def ensure_dir(path: str) -> None:
    """Create directory if it doesn't exist."""
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:
# ==========================================
# BLOCK 2: FACE ALIGNMENT FUNCTIONS
# ==========================================

def get_eye_centers(landmarks, image_shape):
    """
    Extract eye centers from face mesh landmarks.
    Returns left and right eye centers as (x, y) tuples.
    """
    h, w = image_shape[:2]
    
    # MediaPipe landmark indices for eyes
    # Left eye: indices around 33, 133, 159, 145
    # Right eye: indices around 362, 263, 386, 374
    LEFT_EYE_INDICES = [33, 133, 160, 159, 158, 157, 173, 144, 145, 153]
    RIGHT_EYE_INDICES = [362, 263, 387, 386, 385, 384, 398, 373, 374, 380]
    
    # Calculate left eye center
    left_eye_points = []
    for idx in LEFT_EYE_INDICES:
        landmark = landmarks.landmark[idx]
        left_eye_points.append([landmark.x * w, landmark.y * h])
    left_eye_center = np.mean(left_eye_points, axis=0)
    
    # Calculate right eye center
    right_eye_points = []
    for idx in RIGHT_EYE_INDICES:
        landmark = landmarks.landmark[idx]
        right_eye_points.append([landmark.x * w, landmark.y * h])
    right_eye_center = np.mean(right_eye_points, axis=0)
    
    return left_eye_center, right_eye_center


def align_face(image, left_eye, right_eye):
    """
    Align face based on eye positions.
    Rotates the image so eyes are horizontal.
    """
    # Calculate angle between eyes
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))
    
    # Calculate the center point between the eyes
    eyes_center = ((left_eye[0] + right_eye[0]) // 2,
                   (left_eye[1] + right_eye[1]) // 2)
    
    # Get rotation matrix
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D(eyes_center, angle, 1.0)
    
    # Apply rotation
    aligned = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
    
    return aligned, angle


def align_face_advanced(image, landmarks):
    """
    Advanced face alignment using more facial landmarks.
    Uses eyes and nose to create a more accurate alignment.
    """
    h, w = image.shape[:2]
    
    # Get eye centers
    left_eye, right_eye = get_eye_centers(landmarks, image.shape)
    
    # Get nose tip (landmark 1)
    nose_tip = landmarks.landmark[1]
    nose_point = np.array([nose_tip.x * w, nose_tip.y * h])
    
    # Calculate the midpoint between eyes
    eyes_center = (left_eye + right_eye) / 2
    
    # Calculate angle for rotation
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))
    
    # Get rotation matrix
    M = cv2.getRotationMatrix2D(tuple(eyes_center), angle, 1.0)
    
    # Apply rotation
    aligned = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
    
    return aligned, angle


In [3]:
# ==========================================
# BLOCK 3: FACE DETECTION + ALIGNMENT + CROPPING
# ==========================================

def detect_align_and_crop(image_path: str) -> np.ndarray | None:
    """
    Detect face, align it, then crop with margin. Returns BGR numpy array.
    If detection fails and USE_CENTER_FALLBACK=True, returns center crop.
    Otherwise returns None.
    """
    print(f"\n▶ Processing: {image_path}")
    
    # --- Load via PIL (handles EXIF orientation) ---
    try:
        pil_img = Image.open(image_path)
        # Fix orientation (very important for smartphone photos)
        pil_img = ImageOps.exif_transpose(pil_img)
        # Ensure RGB
        pil_img = pil_img.convert("RGB")
        img = np.array(pil_img)
        print(f"  • Loaded image shape (H, W, C): {img.shape}")
    except Exception as e:
        print(f"  ❌ Failed to read image: {e}")
        return None
    
    # Keep original for possible fallback
    orig_img = img.copy()
    orig_h, orig_w = orig_img.shape[:2]
    
    # --- Scale up tiny images ---
    short_side = min(orig_h, orig_w)
    if short_side < MIN_DIM:
        scale = MIN_DIM / float(short_side)
        new_w = int(orig_w * scale)
        new_h = int(orig_h * scale)
        img = cv2.resize(orig_img, (new_w, new_h))
        print(f"  • Upscaled: {orig_w}x{orig_h} → {new_w}x{new_h}")
    
    # --- Scale down huge images ---
    h, w = img.shape[:2]
    if max(h, w) > MAX_DIM:
        scale = MAX_DIM / float(max(h, w))
        new_w = int(w * scale)
        new_h = int(h * scale)
        img = cv2.resize(img, (new_w, new_h))
        print(f"  • Downscaled: {w}x{h} → {new_w}x{new_h}")
        h, w = new_h, new_w
    
    # --- Face Alignment (if enabled) ---
    aligned_img = img.copy()
    alignment_angle = 0
    
    if USE_FACE_ALIGNMENT:
        try:
            # Use Face Mesh for alignment
            results_mesh = face_mesh.process(img)
            
            if results_mesh.multi_face_landmarks:
                landmarks = results_mesh.multi_face_landmarks[0]
                
                # Get eye centers and align
                left_eye, right_eye = get_eye_centers(landmarks, img.shape)
                aligned_img, alignment_angle = align_face(img, left_eye, right_eye)
                
                if abs(alignment_angle) > 0.5:  # Only report if significant rotation
                    print(f"  ✓ Face aligned: rotated {alignment_angle:.2f}°")
            else:
                print(f"  ⚠ Face mesh not detected, skipping alignment")
        except Exception as e:
            print(f"  ⚠ Alignment failed: {e}, using original image")
            aligned_img = img.copy()
    
    # Use aligned image for detection and cropping
    img = aligned_img
    h, w = img.shape[:2]
    
    # --- 1st pass: detect on (possibly aligned) resized image ---
    results = face_detection.process(img)
    
    # --- 2nd pass: try original image if first detection fails ---
    if not results.detections:
        print("  ⚠ No face in processed image, trying original resolution...")
        img = orig_img
        h, w = img.shape[:2]
        results = face_detection.process(img)
    
    # --- If still no detections ---
    if not results.detections:
        print("  ⚠ MediaPipe still failed to detect a face.")
        if not USE_CENTER_FALLBACK:
            return None
        
        # Center crop fallback
        side = int(0.8 * min(h, w))
        cx, cy = w // 2, h // 2
        x1 = max(cx - side // 2, 0)
        y1 = max(cy - side // 2, 0)
        x2 = min(cx + side // 2, w)
        y2 = min(cy + side // 2, h)
        cropped_center = img[y1:y2, x1:x2]
        print(f"  ✅ Used center fallback crop: {cropped_center.shape}")
        return cropped_center
    
    # --- Choose best face ---
    best_det = max(results.detections, key=lambda d: d.score[0])
    bbox = best_det.location_data.relative_bounding_box
    
    # Convert relative bbox to absolute pixel coordinates
    x = int(bbox.xmin * w)
    y = int(bbox.ymin * h)
    box_w = int(bbox.width * w)
    box_h = int(bbox.height * h)
    
    # Sanity check
    x = max(0, x)
    y = max(0, y)
    box_w = max(1, box_w)
    box_h = max(1, box_h)
    
    # --- Apply margin around face box ---
    margin_x = int(box_w * MARGIN_RATIO)
    margin_y = int(box_h * MARGIN_RATIO)
    x1 = max(x - margin_x, 0)
    y1 = max(y - margin_y, 0)
    x2 = min(x + box_w + margin_x, w)
    y2 = min(y + box_h + margin_y, h)
    
    cropped = img[y1:y2, x1:x2]
    print(f"  ✅ Face crop shape (H, W, C): {cropped.shape}")
    
    return cropped

In [4]:
# ==========================================
# BLOCK 4: PROCESS PER PERSON FOLDER
# ==========================================

def process_person_folder(person_name: str) -> None:
    """
    Process all images inside Train/<person_name>
    and save cropped faces to Train_Cropped/<person_name>.
    """
    raw_folder = os.path.join(RAW_DIR, person_name)
    processed_folder = os.path.join(PROCESSED_DIR, person_name)
    ensure_dir(processed_folder)
    
    files = [
        f for f in os.listdir(raw_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png", ".webp", ".heic"))
    ]
    
    total = len(files)
    print(f"\n📁 Folder: {person_name} ({total} files)")
    
    success = 0
    skipped = 0
    
    for idx, filename in enumerate(files, start=1):
        img_path = os.path.join(raw_folder, filename)
        print(f"\n  [{idx}/{total}] File: {filename}")
        
        crop = detect_align_and_crop(img_path)
        
        if crop is None:
            print(f"  ⏭ Skipped: no face detected and no fallback crop.")
            skipped += 1
            continue
        
        # Resize final crop to TARGET_SIZE
        crop = cv2.resize(crop, TARGET_SIZE)
        print(f"  • Resized to: {crop.shape}")
        
        save_path = os.path.join(processed_folder, filename)
        
        # Convert back to PIL Image for saving (no EXIF)
        pil_image = Image.fromarray(crop)
        print(f"  • PIL size (W, H): {pil_image.size}")
        
        # Keep PNG if original is PNG, otherwise JPEG
        if filename.lower().endswith(".png"):
            pil_image.save(save_path, "PNG")
        else:
            pil_image.save(save_path, "JPEG", quality=95)
        
        print(f"  ✅ Saved → {save_path}")
        success += 1
    
    print(f"\n📊 Summary for {person_name}:")
    print(f"  ✔ Success : {success}")
    print(f"  ❌ Skipped : {skipped}")
    print(f"  📦 Total   : {total}")


In [5]:
# ===========================
# BLOCK 5: MAIN RUNNER
# ===========================

def process_all() -> None:
    """Process all subfolders under Train/ as separate persons."""
    if not os.path.exists(RAW_DIR):
        print(f"❌ Folder Train tidak ditemukan di: {RAW_DIR}")
        return
    
    persons = [
        d for d in os.listdir(RAW_DIR)
        if os.path.isdir(os.path.join(RAW_DIR, d))
    ]
    
    if not persons:
        print("⚠ Tidak ada folder mahasiswa di dalam Train/")
        return
    
    print("=" * 60)
    print(" FACE ALIGNMENT + CROPPING PIPELINE (MediaPipe)")
    print("=" * 60)
    print(f"📂 Base dir          : {BASE_DIR}")
    print(f"📂 Input (Train)    : {RAW_DIR}")
    print(f"📂 Output           : {PROCESSED_DIR}")
    print(f"👥 Jumlah folder    : {len(persons)}")
    print(f"🔄 Face Alignment   : {'ENABLED' if USE_FACE_ALIGNMENT else 'DISABLED'}")
    print("=" * 60 + "\n")
    
    ensure_dir(PROCESSED_DIR)
    
    for person in persons:
        process_person_folder(person)
    
    print("\n✅ Selesai! Semua wajah telah diproses dan disimpan di Train_Cropped/")


if __name__ == "__main__":
    process_all()

 FACE ALIGNMENT + CROPPING PIPELINE (MediaPipe)
📂 Base dir          : d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2
📂 Input (Train)    : d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2\./data/Train
📂 Output           : d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2\./data/Train_Cropped
👥 Jumlah folder    : 70
🔄 Face Alignment   : ENABLED


📁 Folder: Abraham Ganda Napitu (4 files)

  [1/4] File: IMG_2138 - Abraham Ganda Napitu.jpeg

▶ Processing: d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2\./data/Train\Abraham Ganda Napitu\IMG_2138 - Abraham Ganda Napitu.jpeg
  • Loaded image shape (H, W, C): (3088, 2316, 3)
  • Downscaled: 2316x3088 → 1200x1599
  ✓ Face aligned: rotated -2.56°
  ✅ Face crop shape (H, W, C): (798, 798, 3)
  • Resized to: (384, 384, 3)
  • PIL size (W, H): (384, 384)
  ✅ Saved → d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2\./data/Train_Cropped\Abraham Ganda Napitu\IMG_2138 - Abraham Ganda Napitu.jpeg

  [2/4] File: IMG_3693 - Abraham Ganda Napitu

d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


  ✓ Face aligned: rotated -2.90°
  ✅ Face crop shape (H, W, C): (946, 945, 3)
  • Resized to: (384, 384, 3)
  • PIL size (W, H): (384, 384)
  ✅ Saved → d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2\./data/Train_Cropped\Abraham Ganda Napitu\IMG_3693 - Abraham Ganda Napitu.jpeg

  [3/4] File: IMG_4951 - Abraham Ganda Napitu.jpeg

▶ Processing: d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2\./data/Train\Abraham Ganda Napitu\IMG_4951 - Abraham Ganda Napitu.jpeg
  • Loaded image shape (H, W, C): (3088, 2316, 3)
  • Downscaled: 2316x3088 → 1200x1599
  ✅ Face crop shape (H, W, C): (793, 793, 3)
  • Resized to: (384, 384, 3)
  • PIL size (W, H): (384, 384)
  ✅ Saved → d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2\./data/Train_Cropped\Abraham Ganda Napitu\IMG_4951 - Abraham Ganda Napitu.jpeg

  [4/4] File: IMG_5357 - Abraham Ganda Napitu.jpeg

▶ Processing: d:\Perkuliahan\Sem 7\Pembelajaran Mendalam\Tubes 2\./data/Train\Abraham Ganda Napitu\IMG_5357 - Abraham Ganda Napitu.jpeg
  •